In [2]:
import google.generativeai as genai

from langchain_chroma import Chroma
from IPython.display import display_markdown

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from pypdf import PdfReader

text = []
reader = PdfReader("sample.pdf")
for page in reader.pages:
    text.append(page.extract_text())

In [5]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [6]:
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

embedder = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash")

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = splitter.create_documents(text)

In [9]:
vector_store = Chroma.from_documents(documents=splits, embedding=embedder)

In [10]:
base_retriever = vector_store.as_retriever(search_kwargs={"k": 1})

In [11]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever, llm=llm, include_original=True
)

In [12]:
system_prompt = """You are a professional support agent that answers questions using text provided from user manuals included below. \
    Be sure to respond in a complete sentences, being comprehensive, and including all relevant information the user might need. \
    However, be sure to break down complicated topics and strike a friendly tone. If the text is irrelevant, you may ignore it. \
    TEXT: {context}
    """

In [13]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [14]:
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=multi_retriever
)

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain.chains import create_retrieval_chain

rag_chain = create_retrieval_chain(compression_retriever, question_answer_chain)

In [17]:
def answer_question(question):
    return rag_chain.invoke(
        {"input": question}
    )["answer"]

In [18]:
response = answer_question("How can I froth milk for my cappuccino?")
display_markdown(response, raw=True)

To froth milk for your cappuccino, follow these steps: 

1. Fill a container (preferably one with a handle to avoid scalding) with about 100 grams of milk for each cappuccino. Remember that the milk will double or triple in volume. 
2. Make sure the cappuccino maker selection ring is positioned downwards in the "CAPPUCCINO" position. 
3. Turn the cappuccino maker slightly outwards to allow containers of any size to be used.
4. Immerse the cappuccino maker in the milk container. 
5. Press the button. 
6. When the light in the button flashes, turn the steam dial to the I position. Steam will be delivered from the cappuccino maker, making the milk frothy. 
7. For a creamier froth, swirl the container slowly, moving it from the bottom upwards. 
8. When you have obtained the required froth, stop steam delivery by turning the steam dial to the O position. 
